# Taller #40: Construcción de un Mini-Sistema de Monitoreo Inteligente
#### Desarrollado por: David Santiago Cruz Hernández

In [ ]:
# !pip install -r requirements.txt

In [1]:
import cv2
import numpy as np
from ultralytics import YOLO
from datetime import datetime
import os
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure

### Configuración Inicial

In [2]:
# Configuración de carpetas
os.makedirs("../capturas", exist_ok=True)
os.makedirs("../logs", exist_ok=True)

# Configuración de rutas
CAPTURE_DIR = "../capturas"
LOG_FILE_PATH = "../logs/eventos.csv"
MODEL_PATH = "../datos/yolo12n.pt"

### Cargar el modelo YOLO

In [3]:
# Inicializar modelo YOLO
model = YOLO(MODEL_PATH)

### Inicializar el archivo de monitoreo

In [4]:
if not os.path.exists(LOG_FILE_PATH):
    with open(LOG_FILE_PATH, "w") as f:
        f.write("timestamp,evento,clase,confianza\n")

# Variables para conteo y gráfico
conteo_objetos = {}
historial_detecciones = []

### Registrar eventos en el archivo de monitoreo

In [5]:
def registrar_evento(evento, clase, confianza):
    """
    Registra un evento en el archivo de monitoreo.
    """
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(LOG_FILE_PATH, "a") as f:
        f.write(f"{timestamp},{evento},{clase},{confianza:.2f}\n")
    print(f"[LOG] {timestamp} - {evento} ({clase}) | Confianza: {confianza:.2f}")

### Dibujar el panel de visualización

In [6]:
def dibujar_panel(frame, conteo, historial, estado="inactivo"):
    """
    Dibuja un panel de visualización con el estado del sistema, conteo de objetos y gráfico de detecciones.
    """
    h, w, _ = frame.shape
    panel = np.zeros((h, 300, 3), dtype=np.uint8)

    # Estado del sistema
    color_estado = (0, 255, 0) if estado == "inactivo" else (0, 0, 255) if estado == "alerta" else (255, 255, 0)
    cv2.putText(panel, f"Estado: {estado}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color_estado, 1)

    # Conteo actual
    y = 60
    for obj, cnt in conteo.items():
        texto = f"{obj}: {cnt}"
        cv2.putText(panel, texto, (10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        y += 25

    # Gráfico
    fig = Figure(figsize=(2.5, 2))
    canvas = FigureCanvas(fig)
    ax = fig.add_subplot(111)
    ax.plot(historial[-20:])
    ax.set_title("Detecciones últimas 20 seg")
    ax.grid(True)
    fig.tight_layout()
    canvas.draw()

    # Extraer imagen del gráfico como array RGBA
    img = np.array(canvas.renderer.buffer_rgba())

    # Convertir RGBA a BGR (OpenCV usa BGR por defecto)
    img = cv2.cvtColor(img, cv2.COLOR_RGBA2BGR)
    img = cv2.resize(img, (295, 200))
    panel[100:300, 5:300] = img

    combined = np.hstack([frame, panel])
    return combined

## Función principal

- ### Inicializar la cámara

In [7]:
def inicializar_camara():
    """
    Inicializa la cámara y devuelve el objeto de captura.
    """
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("[ERROR] No se pudo abrir la cámara.")
        return None
    return cap

- ### Procesar las detecciones

In [8]:
def procesar_detecciones(frame):
    """
    Procesa las detecciones en un frame y devuelve el conteo actual de objetos detectados.
    """
    results = model(frame, verbose=False)
    detecciones = results[0].boxes.data.cpu().numpy()
    current_conteo = {}
    persona_detectada = False
    conf_persona = 0.0

    for *box, conf, cls in detecciones:
        clase = model.names[int(cls)]
        current_conteo[clase] = current_conteo.get(clase, 0) + 1
        if clase == "person":
            persona_detectada = True
            conf_persona = conf
            x1, y1, x2, y2 = map(int, box)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{clase} {conf:.2f}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    return current_conteo, persona_detectada, conf_persona

- ### Actualizar el historial de detecciones

In [9]:
def actualizar_historial(historial, current_conteo, now, last_second):
    """
    Actualiza el historial de detecciones cada segundo.
    """
    if now.second != last_second:
        historial.append(sum(current_conteo.values()))
        return now.second, 0
    return last_second, None

- ### Manejar eventos de detección de personas

In [10]:
def manejar_evento_persona(frame, conf):
    """
    Maneja el evento de detección de una persona, registrando el evento y guardando una captura.
    """
    registrar_evento("Persona detectada", "person", conf)
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    cv2.imwrite(f"{CAPTURE_DIR}/person_{ts}.jpg", frame)
    registrar_evento("Captura guardada", "person", conf)

- ### Mostrar el panel de visualización

In [11]:
def mostrar_panel(frame, current_conteo, historial, estado):
    """
    Muestra el panel de visualización con el estado del sistema, conteo de objetos y gráfico de detecciones.
    """
    vis_frame = dibujar_panel(frame, current_conteo, historial, estado=estado)
    cv2.imshow("YOLOv12 - Detector en tiempo real", vis_frame)

## Función principal del sistema de monitoreo

In [12]:
def main():
    """
    Función principal que inicializa la cámara, procesa las detecciones y muestra el panel de visualización.
    """
    cap = inicializar_camara()
    if cap is None:
        return

    global historial_detecciones
    frame_count = 0
    last_second = datetime.now().second

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Procesar detecciones
        current_conteo, persona_detectada, conf_persona = procesar_detecciones(frame)

        # Actualizar conteo total
        for k, v in current_conteo.items():
            conteo_objetos[k] = conteo_objetos.get(k, 0) + v

        # Registrar evento si corresponde
        if persona_detectada:
            manejar_evento_persona(frame, conf_persona)
            estado = "alerta"
        else:
            estado = "inactivo"

        # Actualizar historial de detecciones
        now = datetime.now()
        nuevo_last_second, reset_frame_count = actualizar_historial(historial_detecciones, current_conteo, now, last_second)
        if reset_frame_count is not None:
            frame_count = 0
            last_second = nuevo_last_second
        frame_count += 1

        # Mostrar panel de visualización
        mostrar_panel(frame, current_conteo, historial_detecciones, estado)

        # Salir si se presiona 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Liberar recursos
    cap.release()
    cv2.destroyAllWindows()



## Ejecutar la función principal

In [ ]:
if __name__ == "__main__":
    main()